# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
def test():
    X = 1
    B = 2
    return X, B

test2 = test()


In [2]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import nltk
import re
import numpy as np
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt','wordnet'])

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect
from sklearn.externals import joblib
from sklearn import svm
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///TH_DBEngine614.db')
df = pd.read_sql_table('df_small', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
genre_counts = df.drop(['id','message','original'], axis=1).groupby('genre').count()
# test = genre_counts.sum()/genre_counts.count()
# test_titles = list(test.index)
# test_titles
# genre_counts
list(genre_counts.loc['direct'])
genre_counts

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
genre,,,,,,,,,,,,,,,,,,,,,
direct,10634,10634,10634,10634,10634,10634,10634,10634,10634,10634,...,10634,10634,10634,10634,10634,10634,10634,10634,10634,10634
news,13036,13036,13036,13036,13036,13036,13036,13036,13036,13036,...,13036,13036,13036,13036,13036,13036,13036,13036,13036,13036
social,2358,2358,2358,2358,2358,2358,2358,2358,2358,2358,...,2358,2358,2358,2358,2358,2358,2358,2358,2358,2358


In [5]:
df.related.unique()

array([ 1.,  0.])

In [6]:
X = df.message.values
Y = df.drop(['message','id','original','genre','child_alone'], axis=1)


In [7]:
Y.columns.values.tolist()

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [9]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipelineRF = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('MOC',MultiOutputClassifier(RandomForestClassifier(n_estimators=10, min_samples_leaf = 1)))
])



pipelineSVM = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('MOC',MultiOutputClassifier(svm.SVC()))
])

pipelineLR = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('MOC',MultiOutputClassifier(LogisticRegression(random_state = 0)))
])

pipelineLR.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'MOC', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'MOC__estimator__C', 'MOC__estimator__class_weight', 'MOC__estimator__dual', 'MOC__estimator__fit_intercept', 'MOC__estimator__intercept_scaling', 'MOC__estimator__max_iter', 'MOC__estimator__multi_class', 'MOC__estimator__n_jobs', 'MOC__estimator__penalty', 'MOC__estimator__random_state', 'MOC__estimator__solver', 'MOC__estimator__tol', 'MOC__estimator__verbose', 'MOC__estimator__warm_start', 'MOC__estimator', 'MOC__n_jobs'])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
Y.sum()


related                   19906.0
request                    4474.0
offer                       118.0
aid_related               10860.0
medical_help               2084.0
medical_products           1313.0
search_and_rescue           724.0
security                    471.0
military                    860.0
water                      1672.0
food                       2923.0
shelter                    2314.0
clothing                    405.0
money                       604.0
missing_people              298.0
refugees                    875.0
death                      1194.0
other_aid                  3446.0
infrastructure_related     1705.0
transport                  1201.0
buildings                  1333.0
electricity                 532.0
tools                       159.0
hospitals                   283.0
shops                       120.0
aid_centers                 309.0
other_infrastructure       1151.0
weather_related            7297.0
floods                     2155.0
storm         

In [12]:
pipelineLR.fit(X_train, y_train)
y_train.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
4889,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14783,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13507,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10772,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17235,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
y_pred = pipelineLR.predict(X_test)



In [14]:

y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)

# y_pred_df['related'].head()
# y_test['related'].head()

for column in y_test.columns:
#     display_results(y_test[column].reset_index(drop=True), y_pred_df[column])
    print("Values for column",column,"\r\n",classification_report(y_test[column].reset_index(drop=True), y_pred_df[column]))


# y_test['related'].reset_index(drop=True)
# y_pred[:,0]

Values for column related 
              precision    recall  f1-score   support

        0.0       0.73      0.42      0.54      1540
        1.0       0.84      0.95      0.89      4967

avg / total       0.82      0.83      0.81      6507

Values for column request 
              precision    recall  f1-score   support

        0.0       0.92      0.98      0.95      5396
        1.0       0.84      0.59      0.69      1111

avg / total       0.91      0.91      0.90      6507

Values for column offer 
              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6477
        1.0       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6507

Values for column aid_related 
              precision    recall  f1-score   support

        0.0       0.80      0.85      0.82      3829
        1.0       0.76      0.69      0.72      2678

avg / total       0.78      0.78      0.78      6507

Values for column medical_

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


This section just looks at all of the different parameters that I have available to choose from to optimize.  I'll optimize the 'MOC__estimator__min_samples_leaf' and 'MOC__estimator__n_estimators' parameters.

In [15]:
pipelineLR.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f65077bb620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('MOC',
   MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
             penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
             verbose=0, warm_start=False),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
parametersRF = {
    'MOC__estimator__min_samples_leaf': [1,2],
    'MOC__estimator__n_estimators':[10, 20]
}

# cv = GridSearchCV(pipelineRF, param_grid = parameters)
# cv.fit(X_train, y_train)

parametersLR = {
    'MOC__estimator__fit_intercept': [True, False]
}

cv = GridSearchCV(pipelineLR, param_grid = parametersLR)
cv.fit(X_train, y_train)




GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...te=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'MOC__estimator__fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [17]:
best_params = cv.best_params_
# print(best_params['MOC__estimator__min_samples_leaf'] + best_params['MOC__estimator__n_estimators'])
best_params

{'MOC__estimator__fit_intercept': True}

In [18]:
best_params = {'MOC__estimator__fit_intercept':best_params['MOC__estimator__fit_intercept']}
print(best_params)

{'MOC__estimator__fit_intercept': True}


In [19]:
pipelineLR.set_params(**best_params)
pipelineLR.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...te=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))])

In [20]:
pipelineLR.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f65077bb620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('MOC',
   MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
             penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
             verbose=0, warm_start=False),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
y_pred2 = pipelineLR.predict(X_test)
y_pred_df2 = pd.DataFrame(y_pred2, columns = y_test.columns)

# y_pred_df['related'].head()
# y_test['related'].head()

for column in y_test.columns:
#     display_results(y_test[column].reset_index(drop=True), y_pred_df[column])
    print("Values for column",column,"\r\n",classification_report(y_test[column].reset_index(drop=True), y_pred_df2[column]))

Values for column related 
              precision    recall  f1-score   support

        0.0       0.73      0.42      0.54      1540
        1.0       0.84      0.95      0.89      4967

avg / total       0.82      0.83      0.81      6507

Values for column request 
              precision    recall  f1-score   support

        0.0       0.92      0.98      0.95      5396
        1.0       0.84      0.59      0.69      1111

avg / total       0.91      0.91      0.90      6507

Values for column offer 
              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6477
        1.0       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6507

Values for column aid_related 
              precision    recall  f1-score   support

        0.0       0.80      0.85      0.82      3829
        1.0       0.76      0.69      0.72      2678

avg / total       0.78      0.78      0.78      6507

Values for column medical_

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('MOC',MultiOutputClassifier(RandomForestClassifier(n_estimators=20, min_samples_leaf = 1)))
])


pipeline2.fit(X_train, y_train)
# y_pred = pipeline.predict(X_test)
# y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)



Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 9. Export your model as a pickle file

In [23]:
filename = 'pipeline2RFC.sav'
pickle.dump(pipeline2, open(filename, 'wb'))


In [24]:
joblib.dump(pipeline2, 'model_dump')

['model_dump']

In [25]:
pickled_model = pickle.load(open(filename, 'rb'))
pickled_model.predict(X_test)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [26]:
joblib_upload = joblib.load('model_dump')
joblib_upload.predict(X_test)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.